In [86]:
import os

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, SymLogNorm 
import numpy as np

from astropy.io import fits
from astropy import wcs
from astropy.coordinates import SkyCoord
import cmocean
from astropy import units as u
from astropy.coordinates import SkyCoord, ICRS, Galactic, FK5
%matplotlib notebook 
ltgreen="#33FF99"

In [87]:
file = 'Merged_Full_Polarization_Rotated.fits'
data = fits.open(file)
w=wcs.WCS(data[0].header)
print(data.info())
Header0 = data[0].header

Filename: Merged_Full_Polarization_Rotated.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  STOKES I      1 PrimaryHDU    1490   (1900, 950)   float64   
  1  ERROR I       1 ImageHDU        19   (1900, 950)   float64   
  2  STOKES Q      1 ImageHDU        19   (1900, 950)   float64   
  3  ERROR Q       1 ImageHDU        19   (1900, 950)   float64   
  4  STOKES U      1 ImageHDU        19   (1900, 950)   float64   
  5  ERROR U       1 ImageHDU        19   (1900, 950)   float64   
  6  IMAGE MASK    1 ImageHDU        60   (1900, 950)   float64   
  7  PERCENT POL    1 ImageHDU        19   (1900, 950)   float64   
  8  DEBIASED PERCENT POL    1 ImageHDU        19   (1900, 950)   float64   
  9  ERROR PERCENT POL    1 ImageHDU        19   (1900, 950)   float64   
 10  POL ANGLE     1 ImageHDU        19   (1900, 950)   float64   
 11  ROTATED POL ANGLE    1 ImageHDU        19   (1900, 950)   float64   
 12  ERROR POL ANGLE    1 ImageHDU        19   (1900, 950)   

In [88]:
Start=SkyCoord('0.05deg','0.085deg')
End=SkyCoord('359.96deg','0.17deg')

x1,y1=w.wcs_world2pix(Start.ra.deg,Start.dec.deg,0)
x2,y2=w.wcs_world2pix(End.ra.deg,End.dec.deg,0)

In [89]:
x1=x1.astype(int)
x2=x2.astype(int)
y1=y1.astype(int)
y2=y2.astype(int)

In [105]:
print(y1,y2,x1,x2)

588 655 994 1065


In [98]:
p_flux = data['DEBIASED POL FLUX'].data # Polarized flux
p_flux = p_flux[y1:y2,x1:x2]
p_flux_err = data['ERROR POL FLUX'].data # Polarized flux error
p_flux_err = p_flux_err[y1:y2,x1:x2]
Stokes_I = data['STOKES I'].data # Stokes I (intensity)
Stokes_I = Stokes_I[y1:y2,x1:x2]

In [99]:
angles = data['ROTATED POL ANGLE'].data # km/s
angles = angles[y1:y2,x1:x2]
angles_err = data['ERROR POL ANGLE'].data # km/s
angles_err = angles_err[y1:y2,x1:x2]

In [100]:
wx, wy = Header0["CRVAL1"], Header0["CRVAL2"]
c = SkyCoord(ra=wx*u.degree, dec=wy*u.degree, frame='icrs')
gal = c.galactic
gal.l.deg, gal.b.deg

(96.51358627207384, -60.2574223649615)

In [101]:
w_new=wcs.WCS(naxis=2)
w_new.wcs.crpix = [68,72]
w_new.wcs.cdelt = [Header0['CDELT1'],Header0['CDELT2']]
w_new.wcs.crval = [gal.l.deg,gal.b.deg]
w_new.wcs.ctype =['GLON-TAN','GLAT-TAN']

In [102]:
im1_head_new = data[0].header
im1_head_new['CRVAL1']=gal.l.deg
im1_head_new['CRVAL2']=gal.b.deg
im1_head_new['CRPIX1']=w_new.wcs.crpix[0]
im1_head_new['CRPIX2']=w_new.wcs.crpix[1]
im1_head_new['CTYPE1']='GLON-TAN'
im1_head_new['CTYPE2']='GLAT-TAN'
im1_head_new['NAXIS1']=950
im1_head_new['NAXIS2']=1900
im1_head_new['CDELT1']=-0.00126388888888888  
im1_head_new['CDELT2']=0.001263888888888889  

In [104]:
hdu1=fits.PrimaryHDU(data=p_flux, header=im1_head_new)
hdu2=fits.PrimaryHDU(data=p_flux_err, header=im1_head_new)
hdu3=fits.PrimaryHDU(data=Stokes_I, header=im1_head_new)
new_hdu1=fits.HDUList([hdu1])
new_hdu2=fits.HDUList([hdu2])
new_hdu3=fits.HDUList([hdu3])
hdu4=fits.PrimaryHDU(data=angles, header=im1_head_new)
hdu5=fits.PrimaryHDU(data=angles_err, header=im1_head_new)
new_hdu4=fits.HDUList([hdu4])
new_hdu5=fits.HDUList([hdu5])


new_hdu1.writeto('p_flux_clip.fits', overwrite=True)
new_hdu2.writeto('p_flux_err_clip.fits', overwrite=True)
new_hdu3.writeto('Stokes_I_clip.fits', overwrite=True)
new_hdu4.writeto('angles_clip.fits', overwrite=True)
new_hdu5.writeto('angles_err_clip.fits', overwrite=True)